**<font color=skyblue>示範 PyTorch 的淺度機器學習（典型的 Neural Network）應用在AT&T 人臉辨識的訓練與測試過程</font>**

*<font color=blue>Load data and prepare for PyTorch Tensor</font>*

Tensors are used in machine learning to represent data and model parameters. They are used to encode the inputs and outputs of a model as well as the model’s parameters. In PyTorch, tensors are used to represent the inputs and outputs of a model as well as the model’s parameters.

Torch is a scientific computing framework with wide support for machine learning algorithms that puts GPUs first.

In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

df = pd.read_csv(r'D:\ys\B4_ShallowML\ShallowML\ClassData\face_data.csv')
n_persons = df['target'].nunique() 
X = np.array(df.drop('target', axis=1)) # 400 x 4096
y = np.array(df['target'])

test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) # deafult test_size=0.25

# prepare data for PyTorch Tensor
X_train = torch.from_numpy(X_train).float() # convert to float tensor
y_train = torch.from_numpy(y_train).float() # 
train_dataset = TensorDataset(X_train, y_train) # create your datset
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
test_dataset = TensorDataset(X_test, y_test) # create your datset

# create dataloader for PyTorch
batch_size = 32 # 32, 64, 128, 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # convert to dataloader
test_loader = DataLoader(test_dataset, batch_size=len(X_test), shuffle=False) # 不設定 batch_size, 會把所有資料放在一起

<hr>

In [2]:
A = torch.reshape(X_train, (X_train.shape[0], 1, 64, 64)) # 400 x 1 x 64 x 64 for CNN

<hr>

*<font color=blue>Set up NN  model</font>*

<hr>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu" # run faster than cuda in some cases
print("Using {} device".format(device))

# Create a neural network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(64*64, 512), # image length 64x64=4096,  fully connected layer
            nn.ReLU(), # you may want to take ReLU out to see what happen
            nn.Linear(512, 128), # second hidden layer
            nn.ReLU(),
            nn.Linear(128, 40) # 40 classes,  fully connected layer
            # nn.Softmax()
        )
    # Specify how data will pass through this model
    def forward(self, x):
        # out = self.mlp(x) 

        # Apply softmax to x here~
        x = self.mlp(x) # pass through the model
        out = F.log_softmax(x, dim=1) # it’s faster and has better numerical propertie than softmax
        # out = F.softmax(x, dim=1)
        return out

# define model, optimizer, loss function
model = MLP().to(device) # start an instance
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # default lreaning rate=1e-3
loss_fun = nn.CrossEntropyLoss() # define loss function

print(model)

Using cpu device
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=40, bias=True)
  )
)


<hr>

*<font color=blue>Test the forward process of the nn model</font>*

In [4]:
input = torch.randn(1, 64 * 64) 
m1 = nn.Linear(64*64, 512)
output = m1(input)
print(output.size())
output = F.relu(output)
print(output.size())
m2 = nn.Linear(512, 40)
output = m2(output)
print(output.size())
# output = F.log_softmax(output, dim=1)
output = F.softmax(output, dim=1)
print(output.size())
print(output)
print(output.argmax(dim=1))

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 40])
torch.Size([1, 40])
tensor([[0.0429, 0.0240, 0.0200, 0.0200, 0.0310, 0.0216, 0.0294, 0.0215, 0.0174,
         0.0197, 0.0263, 0.0216, 0.0289, 0.0280, 0.0158, 0.0205, 0.0253, 0.0301,
         0.0305, 0.0284, 0.0318, 0.0295, 0.0207, 0.0213, 0.0351, 0.0253, 0.0219,
         0.0314, 0.0296, 0.0258, 0.0177, 0.0268, 0.0269, 0.0212, 0.0207, 0.0190,
         0.0146, 0.0196, 0.0365, 0.0216]], grad_fn=<SoftmaxBackward0>)
tensor([0])


In [5]:
output = nn.Linear(64*64, 512)(X_train)
print(output.shape)
pr = nn.ReLU()(output)
p = nn.Linear(512, 40)(pr)
print(p.shape)

torch.Size([280, 512])
torch.Size([280, 40])


<hr>

*<font color=yellow>Training</font>*
<hr>

In [6]:
from tqdm import tqdm

epochs = 50 # Repeat the whole dataset epochs times
model.train() # Sets the module in training mode. The training model allow the parameters to be updated during backpropagation.
for epoch in range(epochs):
# for epoch in tqdm(range(epochs)):
    trainAcc = 0
    samples = 0
    losses = []
    for batch_num, input_data in enumerate(train_loader):
    # for batch_num, input_data in tqdm(enumerate(train_loader), total=len(train_loader)):
        
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device)

        # perform training based on the backpropagation
        y_pre = model(x) # predict y
        loss = loss_fun(y_pre, y.long()) # the loss function nn.CrossEntropyLoss()
        losses.append(loss.item())

        optimizer.zero_grad() # Zeros the gradients accumulated from the previous batch/step of the model
        loss.backward() # Performs backpropagation and calculates the gradients
        optimizer.step() # Updates the weights in our neural network based on the results of backpropagation
        
        # Record the training accuracy for each batch
        trainAcc += (y_pre.argmax(dim=1) == y).sum().item() # comparison
        samples += y.size(0)
        if batch_num % 4 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f | train accuracy %.4f' % (epoch, sum(losses)/len(losses), trainAcc/samples))

print('Finished ... Loss %7.4f | train accuracy %.4f' % (sum(losses)/len(losses), trainAcc/samples))

	Epoch 0 | Batch 0 | Loss   3.68
	Epoch 0 | Batch 4 | Loss   3.93
	Epoch 0 | Batch 8 | Loss   3.80
Epoch 0 | Loss   3.81 | train accuracy 0.0143
	Epoch 1 | Batch 0 | Loss   3.68
	Epoch 1 | Batch 4 | Loss   3.79
	Epoch 1 | Batch 8 | Loss   3.66
Epoch 1 | Loss   3.72 | train accuracy 0.0179
	Epoch 2 | Batch 0 | Loss   3.68
	Epoch 2 | Batch 4 | Loss   3.63
	Epoch 2 | Batch 8 | Loss   3.72
Epoch 2 | Loss   3.65 | train accuracy 0.0357
	Epoch 3 | Batch 0 | Loss   3.61
	Epoch 3 | Batch 4 | Loss   3.62
	Epoch 3 | Batch 8 | Loss   3.64
Epoch 3 | Loss   3.63 | train accuracy 0.0679
	Epoch 4 | Batch 0 | Loss   3.61
	Epoch 4 | Batch 4 | Loss   3.59
	Epoch 4 | Batch 8 | Loss   3.55
Epoch 4 | Loss   3.59 | train accuracy 0.0286
	Epoch 5 | Batch 0 | Loss   3.51
	Epoch 5 | Batch 4 | Loss   3.58
	Epoch 5 | Batch 8 | Loss   3.54
Epoch 5 | Loss   3.55 | train accuracy 0.0571
	Epoch 6 | Batch 0 | Loss   3.51
	Epoch 6 | Batch 4 | Loss   3.45
	Epoch 6 | Batch 8 | Loss   3.50
Epoch 6 | Loss   3.51 | train a

<hr>

*<font color=blue>Testing (1)</font>*

Compute test accuracy by batch

In [8]:
model.eval() 
testAcc = 0
samples = 0
with torch.no_grad():
    for x, y_truth in test_loader:
        x = x.to(device).float()
        y_truth = y_truth.to(device)
        y_pre = model(x).argmax(dim=1) # the predictions for the batch
        testAcc += (y_pre == y_truth).sum().item() # comparison
        samples += y_truth.size(0)

    print('Test Accuracy:{:.3f}'.format(testAcc/samples))


Test Accuracy:0.767


*<font color=blue>Testing (2)</font>*

Compute the test accuracy and record the result for each test data

In [9]:
import csv

# use eval() in conjunction with a torch.no_grad() context, 
# meaning that gradient computation is turned off in evaluation mode
model.eval() 
testAcc = 0
samples = 0

with open('mlp_att.csv', 'w') as f:
    fieldnames = ['ImageId', 'Label', 'Ground_Truth']
    writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
    writer.writeheader()
    image_id = 1

    with torch.no_grad():
        for x, y_truth in test_loader:
            x = x.to(device).float()
            y_truth = y_truth.to(device).long()
            yIdx = 0
            y_pre = model(x).argmax(dim=1) # the predictions for the batch
            testAcc += (y_pre == y_truth).sum().item() # comparison
            samples += y_truth.size(0)
            for y in y_pre:
                writer.writerow({fieldnames[0]: image_id,fieldnames[1]: y.item(), fieldnames[2]: y_truth[yIdx].item()})
                image_id += 1
                yIdx += 1

        print('Test Accuracy:{:.3f}'.format(testAcc/samples))

Test Accuracy:0.767
